In [1]:
import pandas as pd
import jieba
import re
import numpy as np
import gensim
from keras_self_attention import SeqSelfAttention
import pickle
from keras import regularizers

In [2]:
maxlen=75 #  在75个词后截断评论
max_words=50000 # 考虑前10000个出现的词 发现有134658个word_index，故调整为50000
embedding_dim=100

In [3]:
data=np.load("./data_75.npy")

In [4]:
from keras import layers
from keras import models
def Build_model(): # 第二层变为128
    model=models.Sequential()
    model.add(layers.Embedding(Embedding_matrix.shape[0],Embedding_matrix.shape[1],input_length=maxlen))
    # model.add(SeqSelfAttention())
    model.add(layers.LSTM(128,kernel_regularizer=regularizers.l2(0.001),return_sequences=True))
    # model.add(layers.LSTM(128,return_sequences=True,kernel_regularizer=regularizers.l2(0.001)))
    model.add(layers.LSTM(128,kernel_regularizer=regularizers.l2(0.001)))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(128,activation='relu',kernel_regularizer=regularizers.l2(0.001)))
    model.add(layers.Dense(64,activation='relu',kernel_regularizer=regularizers.l2(0.001)))
    model.add(layers.Dense(1,activation='sigmoid'))
    model.layers[0].set_weights([Embedding_matrix])
    model.layers[0].trainable=False
    model.summary()
    return model

In [5]:
labels=np.load("./labels.npy")

In [6]:
Embedding_matrix=np.load("./Embedding/Embedding_matrix_200.npy")

In [7]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint( './model_{epoch:03d}.hdf5',
                                   verbose=1, save_weights_only=True,monitor='val_acc',period=1)

In [8]:
model=Build_model()
model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['acc'])
history=model.fit(data,
          labels,
          epochs=20,
          batch_size=128,
          validation_split=0.2,
                 callbacks=[checkpointer])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 75, 200)           10000000  
_________________________________________________________________
lstm (LSTM)                  (None, 75, 128)           168448    
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               16512     
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 6